In [21]:
import asyncio
from ib_insync import *
ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=9)  # ✅ FIXED

<IB connected to 127.0.0.1:7497 clientId=9>

In [22]:

import pandas as pd
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time


In [23]:

tickers = ['AAPL']
num_days = 60
num_years =25
months_batches = 6
output_dir = 'ibkr_1min_data_by_insync/'
os.makedirs(output_dir, exist_ok=True)

# Async function to fetch data
async def fetch_1min_data(ticker, date):
    contract = Stock(ticker, 'SMART', 'USD')
    bars = await ib.reqHistoricalDataAsync(
        contract,
        endDateTime=date.strftime("%Y%m%d %H:%M:%S"),
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1
    )
    return util.df(bars)

# Wrapper function to run the loop
async def fetch_all():
    current_date = datetime.now()
    for ticker in tickers:
        print(f"Fetching data for {ticker}...")
        all_data = []
        for year in range(num_years):
            #need to iteration of 60 days batch
            for batch in range(months_batches):
                start_time = time.perf_counter()
                for i in range(num_days) :
                    target_date = current_date - relativedelta(years=year) - relativedelta(months=2*batch) -timedelta(days=i)
                    try:
                        df = await fetch_1min_data(ticker, target_date)
                        if not df.empty:
                            all_data.append(df)
                            print(f"{ticker} - {target_date.date()} ✅")
                        else:
                            print(f"{ticker} - {target_date.date()} ⛔ No data")
                    except Exception as e:
                        print(f"{ticker} - {target_date.date()} ❌ Error: {e}")
                    await asyncio.sleep(0.6)  # Avoid rate limits
                end_time = time.perf_counter()
                delta = end_time - start_time
                await asyncio.sleep(timedelta(minutes=10)-delta)

        if all_data:
            final_df = pd.concat(all_data)
            final_df.to_csv(f"{output_dir}{ticker}_1min.csv", index=False)
            print(f"Saved {ticker} to CSV ✅")
        else:
            print(f"No data collected for {ticker} ❗")


await fetch_all()

Fetching data for AAPL...
AAPL - 2023-07-02 ✅
AAPL - 2023-07-01 ✅
AAPL - 2023-06-30 ✅
AAPL - 2023-06-29 ✅
AAPL - 2023-06-28 ✅
AAPL - 2023-06-27 ✅
AAPL - 2023-06-26 ✅
AAPL - 2023-06-25 ✅
AAPL - 2023-06-24 ✅
AAPL - 2023-06-23 ✅
AAPL - 2023-06-22 ✅
AAPL - 2023-06-21 ✅
AAPL - 2023-06-20 ✅
AAPL - 2023-06-19 ✅
AAPL - 2023-06-18 ✅
AAPL - 2023-06-17 ✅
AAPL - 2023-06-16 ✅
AAPL - 2023-06-15 ✅
AAPL - 2023-06-14 ✅
AAPL - 2023-06-13 ✅
AAPL - 2023-06-12 ✅
AAPL - 2023-06-11 ✅
AAPL - 2023-06-10 ✅
AAPL - 2023-06-09 ✅
AAPL - 2023-06-08 ✅
AAPL - 2023-06-07 ✅
AAPL - 2023-06-06 ✅
AAPL - 2023-06-05 ✅
AAPL - 2023-06-04 ✅
AAPL - 2023-06-03 ✅
AAPL - 2023-06-02 ✅
AAPL - 2023-06-01 ✅
AAPL - 2023-05-31 ✅
AAPL - 2023-05-30 ✅
AAPL - 2023-05-29 ✅
AAPL - 2023-05-28 ✅
AAPL - 2023-05-27 ✅
AAPL - 2023-05-26 ✅
AAPL - 2023-05-25 ✅
AAPL - 2023-05-24 ✅
AAPL - 2023-05-23 ✅
AAPL - 2023-05-22 ✅
AAPL - 2023-05-21 ✅
AAPL - 2023-05-20 ✅
AAPL - 2023-05-19 ✅
AAPL - 2023-05-18 ✅
AAPL - 2023-05-17 ✅
AAPL - 2023-05-16 ✅
AAPL - 2023-05

In [25]:
ib.disconnect()